In [2]:
import agate
import warnings
from pathlib import Path
from shutil import copyfile
import os
from decimal import *
from more_itertools import unique_everseen
import csv

warnings.filterwarnings(action='ignore')

g_table_geos = agate.Table.from_csv('../data/county-geos.csv')

def is_float(string):
    try:
        # True if string is a number contains a dot
        return float(string) and '.' in string
    # String is not a number
    except ValueError:
        return False


def is_number(value):
    try:
        if value is None:
            return False
        elif value.isdigit() or is_float(value):
            return True
    except ValueError:
        return False
# Get percent of filters in the table
# The filters and table are parameters
# f is short for filter
def perc_calc(table, f_col, fs):
  try:
    f_sum = 0
    f_tables = []
    f_string = ""
    
    for i in fs:
        f_table = table.aggregate( agate.Count(f_col, i) )
        f_tables.append(f_table)
        f_sum += f_table
    
    for index,i in enumerate(fs):
        f_perc = round(f_tables[index] / f_sum * 100, 2)
        f_string += str(i) + ': ' + str(f_perc) + ', '
    
    return f_string

  except ValueError:
    return False

def perc_global(table_one, table_two, classify_column, classify_column_two, perc_column, filter_column, filters, limit):
    try:
        # Join with geos
        table_geos = table_one.join(g_table_geos, classify_column, 'county', inner=True)
        table_defs_geos = table_two.join(g_table_geos, classify_column_two, 'county', inner=True)
        table_join = table_geos.join(table_defs_geos, 'geo', 'geo')
        
        # If something didn't join right, print it out
        if table_join:
            table_join_null = table_join.where(lambda row: row[filter_column] == None)
            
            if table_join_null:
                print("Not everything joined. Table saved to tmp dir.")
                table_join.to_csv('../tmp/table-not-joined.csv')
        
        # Select only a few rows, get only integer or floats
        # And order
        table_filter = table_join.select((classify_column, perc_column, filter_column))
        table_order = table_filter.order_by(lambda row: row[perc_column])
        table_bottom = table_order.limit(limit)
        
        lower = filter_column.replace(' ', '-').lower()
        print('Entire table')
        print( perc_calc(table_order, filter_column, filters) )
        table_order.to_csv('../edits/' + lower + '/01-' + lower + '.csv')
        print('-')
        print( 'Bottom ' + str(limit) )
        print( perc_calc(table_bottom, filter_column, filters) )
        table_bottom.to_csv('../edits/' + lower + '/01-' + lower + '-bottom.csv')
    
    except ValueError:
        return False
    
def join_tables(fp, classify_column, classify_column_two, rename_columns):            
    try:
        table = agate.Table.from_csv(fp, encoding='utf-8-sig')
        
        # Not our first time through
        # Join tables
        if Path(g_table_merge_fp).is_file():
            print("file exists")
            # Take our global table
            # And our table getting passed through the function
            # give them both geo ids
            # then join to make one big, beautiful table
            table_one = agate.Table.from_csv(g_table_merge_fp, encoding='utf-8-sig')
            
            table_one.to_csv('../tmp/table-one.csv')
            table.to_csv('../tmp/table.csv')
            
            # table_one.print_table()
            table_geos = table_one.join(g_table_geos, classify_column, 'county',
                                    inner=True)
            
            # table_geos.select(('geo')).print_table()
            table_geos_two = table.join(g_table_geos, classify_column_two, 'county',
                                    inner=True)
            
            table_geos.to_csv('../tmp/table-geos.csv')
            table_geos_two.to_csv('../tmp/table-geos-two.csv')
            
            table_join = table_geos.join(table_geos_two, 'geo', 'geo')
            # If something didn't join right, print it out
            if table_join:
                table_join_null = table_join.where(lambda row: row[classify_column] == None)

                if table_join_null:
                    print("Not everything joined. Table saved to tmp dir.")
                    table_join.to_csv('../tmp/table-not-joined.csv')


            if rename_columns:
                table_join_final = table_join.rename(column_names = rename_columns)
            else:
                table_join_final = table_join
            
            print("saving joined table")
            table_join_final.exclude(
                ['geo2', 'geo2_2', 'geo2_3', 'county2', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']
            ).to_csv(g_table_merge_fp)
        # First time through
        # Don't join tables
        else:
            if rename_columns:
                table_final = table.rename(column_names = rename_columns)
            else:
                table_final = table
                
            print("file doesn't exist")
            table_final.to_csv(g_table_merge_fp)
        
    except ValueError:
        return False
    
class AddBuckets(agate.Computation):
    def get_computed_data_type(self, table):
        return agate.Number()
    
    def run(self, table):
        new_column = []
        
        for index, row in enumerate(table.rows):
            match = False
            
            for i in filters_all:
                split = i.split('-')
                row_data = int(row[filter_column_og])
                
                if row_data >= int(split[0]) and row_data < ( int(split[1]) + 1):
                    new_column.append(i)
                    match = True
            
            if not match:
                print(row[filter_column_og])
                
        return new_column
    
class AddPercent(agate.Computation):
    def get_computed_data_type(self, table):
        return agate.Number()
    
    def run(self, table):
        new_column = []
        
        for index, row in enumerate(table.rows):
            perc = round(row[table_two_perc] / row[table_two_perc_total] * 100, 2)
            
            new_column.append(perc)
                
        return new_column

In [24]:
table_one = agate.Table.from_csv('../data/prenatal/prenatal-county-historical-10-14.csv')
table_two = agate.Table.from_csv('../data/prenatal/rural-urban-defs.csv')
classify_column = 'county'
classify_column_two = 'county'
perc_column = 'prenatal_percent_2010_2014'
filter_column =  'Urbanization Status'
filters = ['Rural', 'Urban']
limit = 20

perc_global(table_one, table_two, classify_column, classify_column_two, perc_column, filter_column, filters, limit)

Entire table
Rural: 67.72, Urban: 32.28, 
-
Bottom 20
Rural: 85.0, Urban: 15.0, 


In [25]:
table_one = agate.Table.from_csv('../data/prenatal/prenatal-county-historical-10-14.csv')
table_two = agate.Table.from_csv('../data/prenatal/rural-urban-defs.csv')
classify_column = 'county'
classify_column_two = 'county'
perc_column = 'prenatal_percent_2010_2014'
filter_column =  'Border Status'
filters =  ['Border', 'Non-Border']
limit = 20

perc_global(table_one, table_two, classify_column, classify_column_two, perc_column, filter_column, filters, limit)

Entire table
Border: 12.6, Non-Border: 87.4, 
-
Bottom 20
Border: 30.0, Non-Border: 70.0, 


In [11]:
class AddSum(agate.Computation):
    def get_computed_data_type(self, table):
        return agate.Number()
    
    def run(self, table):
        new_column = []
        
        for index, row in enumerate(table.rows):
            col = row['number_2010'] + row['number_2011'] + row['number_2012'] + row['number_2013'] + row['number_2014']
            
            new_column.append(col)
                
        return new_column
    
class AddEligible(agate.Computation):
    def get_computed_data_type(self, table):
        return agate.Number()
    
    def run(self, table):
        new_column = []
        
        for index, row in enumerate(table.rows):
            if is_number(row['prenatal_percent_2010_2014']):
                col = round( float(row['prenatal_sum_2010_2014']) / (1 - ( (100 -  float(row['prenatal_percent_2010_2014']) ) /100) ) )
            else:
                col = '-'
            
            new_column.append(col)
                
        return new_column
    
table_one = agate.Table.from_csv('../data/prenatal/prenatal-county-historical-10-14.csv')
table_add_one = table_one.compute([
    ( 'prenatal_sum_2010_2014', AddSum() )
])
table_add_one.to_csv('../edits/prenatal/01-sum.csv')

table_add_two = table_add_one.compute([
    ( 'prenatal_eligible_2010_2014', AddEligible() )
])
table_add_two.to_csv('../edits/prenatal/02-sum-eligible.csv')

In [13]:
years = ['2010', '2011', '2012', '2013', '2014']

g_table_merge_fp = '../edits/prenatal/03-races.csv'

# Remove file so we can create new one
if Path(g_table_merge_fp).is_file():
    os.remove(g_table_merge_fp)

i = 1
for year in years:
    print(year)
    
    fp = '../data/prenatal/raw/' + year + '-county.csv'
    columns = False
    
    if i == 1:
        classify_column = 'Geographic Area'
        classify_column_two = 'Geographic Area'
        i = ''
    else:
        classify_column = 'county'
        classify_column_two = 'Geographic Area'
        i = '2'
    
    rename_columns = {'Geographic Area': 'county', 'Total Number': 'total_number_' + year, 'Total Percent': 'total_percent_' + year, 'White Number': 'white_number_' + year, 'White Percent': 'white_percent_' + year, 'Black Number': 'black_number_' + year, 'Black Percent': 'black_percent_' + year, 'Hispanic Number': 'hispanic_number_' + year, 'Hispanic Percent': 'hispanic_percent_' + year}
    
    if i == '':
        i = 1
    else:
        del rename_columns['Geographic Area']
        i = int(i)
        
    join_tables(fp, classify_column, classify_column_two, rename_columns)
    
    i += 1

2010
file doesn't exist
2011
file exists
saving joined table
2012
file exists
saving joined table
2013
file exists
saving joined table
2014
file exists
saving joined table


In [15]:
class AddSumTwo(agate.Computation):
    def get_computed_data_type(self, table):
        return agate.Number()
    
    def run(self, table):
        new_column = []
        years = ['2010', '2011', '2012', '2013', '2014']
        
        for index, row in enumerate(table.rows):
            c_sum = 0
            for year in years:
                c_sum += row[g_race + '_number_' + year]
            
            new_column.append(c_sum)
                
        return new_column
    
table_one = agate.Table.from_csv('../edits/prenatal/03-races.csv')
races = ['total', 'white', 'black', 'hispanic']
g_race = races[0]

table_two = table_one.compute([
    ( g_race + '_sum_2010_2014', AddSumTwo() )
])

for index, race in enumerate(races):
    if index > 0:
        g_race = race
    
        table_two = table_two.compute([
            ( g_race + '_sum_2010_2014', AddSumTwo() )
        ])
    
table_two.to_csv('../edits/prenatal/04-races-sum.csv')

In [55]:
# class AddEligibleTwo(agate.Computation):
#     def get_computed_data_type(self, table):
#         return agate.Number()
    
#     def run(self, table):
#         new_column = []
        
#         years = ['2010', '2011', '2012', '2013', '2014']
        
#         for index, row in enumerate(table.rows):
#             c_sum = 0
#             for year in years:
#                 if row[g_race + '_percent_' + year] != '*' and row[g_race + '_percent_' + year] != '-':
#                     c_sum += round( float(row[g_race + '_number_' + year]) / (1 - ( (100 - float(row[g_race + '_percent_' + year]) ) /100) ) )
            
#             new_column.append(c_sum)
                
#         return new_column

# table_one = agate.Table.from_csv('../edits/prenatal/04-races-sum.csv')
# races = ['total', 'white', 'black', 'hispanic']
# g_race = races[0]

# table_two = table_one.compute([
#     ( g_race + '_eligible_2010_2014', AddEligibleTwo() )
# ])

# for index, race in enumerate(races):
#     if index > 0:
#         g_race = race
    
#         table_two = table_two.compute([
#             ( g_race + '_eligible_2010_2014', AddEligibleTwo() )
#         ])
    
# table_two.to_csv('../edits/prenatal/05-races-eligible.csv')

In [61]:
# class AddPercent(agate.Computation):
#     def get_computed_data_type(self, table):
#         return agate.Number()
    
#     def run(self, table):
#         new_column = []
        
#         for index, row in enumerate(table.rows):
#             row_race = int(row[g_race + '_sum_2010_2014'])
#             row_total = int(row[g_race + '_eligible_2010_2014'])
#             if row_race == 0 or row_total == 0:
#                 perc = 0
#             else:
#                 perc = round(row_race / row_total * 100, 2)
            
#             new_column.append(perc)
                
#         return new_column
    
# table_one = agate.Table.from_csv('../edits/prenatal/05-races-eligible.csv')
# races = ['total', 'white', 'black', 'hispanic']
# g_race = races[0]

# table_two = table_one.compute([
#     ( g_race + '_percent_2010_2014', AddPercent() )
# ])

# for index, race in enumerate(races):
#     if index > 0:
#         g_race = race
    
#         table_two = table_two.compute([
#             ( g_race + '_percent_2010_2014', AddPercent() )
#         ])
        
# table_two.to_csv('../edits/prenatal/06-races-percent.csv')

In [ ]:
# table_join = agate.Table.from_csv('../edits/02-mortality-prenatal-urbanization-10-14-population.csv')
# table_two = table_one.join(table_join, 'geo', 'geo', inner=True)

# table_two.to_csv('../edits/03-prenatal-race-full.csv')

# .select(('county', 'geo', 'geo_two', 'GISJOIN', 'total_sum_2010_2014', 'white_sum_2010_2014', 'black_sum_2010_2014', 'hispanic_sum_2010_2014'))